<a href="https://colab.research.google.com/github/shumshersubashgautam/Deep-NLP/blob/main/Text_EDA%2C_RNN%2C_LSTM%2C_GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
# from google.colab import files

# files.upload()

In [2]:
!ls -lha kaggle.json

-rw-r--r-- 1 root root 74 May  1 08:13 kaggle.json


In [3]:
!pip install -q kaggle

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [5]:
!chmod 600 /root/.kaggle/kaggle.json

In [6]:
!pwd

/content


In [10]:
!kaggle datasets download -d samdeeplearning/deepnlp

  0% 0.00/234k [00:00<?, ?B/s]
100% 234k/234k [00:00<00:00, 128MB/s]


In [11]:
!unzip /content/deepnlp.zip

Archive:  /content/deepnlp.zip
  inflating: Sheet_1.csv             
  inflating: Sheet_2.csv             


In [71]:
import pandas as pd
import numpy as np

# for handling text
import string
import nltk
import seaborn as sns
%matplotlib inline
import sklearn
import regex as re
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import time
from termcolor import colored
stop_words = set(stopwords.words('english'))

# for plots
import matplotlib as plty
import matplotlib.pyplot as plt
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.express as px
%matplotlib inline

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score

pd.set_option('display.max_colwidth', None)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [72]:
df = pd.read_csv("/content/Sheet_1.csv")
df.dropna(inplace=True, axis=1 )
df.head(4)

,response_id,class,response_text
0,response_1,not_flagged,I try and avoid this sort of conflict
1,response_2,flagged,Had a friend open up to me about his mental addiction to weed and how it was taking over his life and making him depressed
2,response_3,flagged,"I saved a girl from suicide once. She was going to swallow a bunch of pills and I talked her out of it in a very calm, loving way."
3,response_4,not_flagged,i cant think of one really...i think i may have indirectly


In [74]:
fig = px.histogram(df, x='class', title='distribution of records')
fig.show()

In [75]:
def tokenize_text(df, input_col='response_text', output_col="word_list"):
    """
    takes a dataset and name of a column
    then tokenizes the text in the column of that dataset
    """
    df.loc[:, output_col] = df.loc[:, input_col].apply(lambda t: word_tokenize(t))
    return df
    
df = tokenize_text(df)

In [76]:
def remove_stopwords(list_of_words):
    filtered_list = [w for w in list_of_words if not w.lower() in stop_words]
    return filtered_list

def check_punct(list_of_words):
    """
    look at the tokenized text. if there was any punctuation, it is redundant.
    """
    filtered_list = []
    for word in list_of_words:
        if re.findall("[()!><.,`?']", word):
            pass
        else: filtered_list.append(word)
        
    return filtered_list

def clean(df, input_col='word_list', output_col="cleaned_word_list"):
    
    """
    takes a column of textual data 
    outputs a df with cleaned text attached
    """
    texts = df.loc[:, input_col]
    word_list = []
    
    for text in texts: 
        t = remove_stopwords(text)
        t = check_punct(t)
        word_list.append(t)
        
    df.loc[:, output_col] = word_list
    
    return df

df = clean(df)

In [77]:
def try_this(t):
    try: word_tokenize(t)
    except: print(t)

In [78]:
df.head(1)

,response_id,class,response_text,word_list,cleaned_word_list
0,response_1,not_flagged,I try and avoid this sort of conflict,"[I, try, and, avoid, this, sort, of, conflict]","[try, avoid, sort, conflict]"


In [79]:
def join_tokens(df, tokens_arrays_col):
    return [" ".join(df.loc[i, tokens_arrays_col]) for i in range(len(df))]

df.loc[:, 'cleaned_text'] = join_tokens(df, 'cleaned_word_list')

In [80]:
df_f = df[df['class'] == 'flagged']
df_n = df[df['class'] == 'not_flagged']

In [81]:
def count_words(tokens_arrays):
    """
    gets a dictionary and counts the values
    output: a sorted dict
    note: you can also use a bag of words package to do this
    """
    count_dict = {}
    for array_ in tokens_arrays:
        for word in array_:
            try: count_dict[word] +=1
            except: count_dict[word] = 1

    # sort 
    sorted_count_dict = {k:v for k,v in sorted(count_dict.items(), key=lambda item: item[1], reverse=True)}
    
    return sorted_count_dict

In [82]:
def get_n_key_and_value(n, dict_):

    """
    get the first - most frequent and important -
    words of dictionary 
    """
    keys = [k for (k, v) in dict_.items()][:n]
    values = [v for (k, v) in dict_.items()][:n]

    return keys, values

In [83]:
def convert_tokens_list_to_freq_df(tokens_arrays, n=-1):
    """
    gets the array of tokenized sentences
    output: a sorted dataframe with two cols
    the words and their frequency
    """

    dict_ = count_words(tokens_arrays)
    keys, values = get_n_key_and_value(n, dict_)

    df = pd.DataFrame({'words': keys, 'freq': values})

    return df

df_f_words_freq = convert_tokens_list_to_freq_df(df_f.cleaned_word_list)
df_n_words_freq = convert_tokens_list_to_freq_df(df_n.cleaned_word_list)

In [84]:
df_f_words_freq.head()

,words,freq
0,friend,13
1,people,10
2,friends,9
3,would,7
4,going,6


In [85]:
def get_tfidf_words_and_array(text_arrays):

    vectorizer = TfidfVectorizer()
    transformed_data = vectorizer.fit_transform(text_arrays).toarray()
    words = vectorizer.get_feature_names_out()
    
    return transformed_data, words

In [86]:
def create_tfidf_df(text_arrays):
    """
    gets the df, converts it into tfidf arrays and words
    then puts them in a dataset
    """

    transformed_data, words = get_tfidf_words_and_array(text_arrays)

    df = pd.DataFrame(data=transformed_data, columns=words).sum().reset_index()

    col_names = ['words', 'tfidf_score_sum']
    default_col_names = df.columns

    # rename whatever the df cols are called to the col_names
    df.rename(columns={default_col_names[i]:col_names[i] for i in range(len(col_names))}, inplace=True)

    return df

In [87]:
tfidf_scores_f = create_tfidf_df(df_f.loc[:, 'cleaned_text'])
tfidf_scores_n = create_tfidf_df(df_n.loc[:, 'cleaned_text'])

In [88]:
def merge(df_1, df_2, on='words'):
    return pd.merge(left=df_1, right=df_2, on=on, how='left')

In [89]:
f_words_df = merge(df_f_words_freq, tfidf_scores_f)
n_words_df = merge(df_n_words_freq, tfidf_scores_n)

In [90]:
f_words_df.head(4)

,words,freq,tfidf_score_sum
0,friend,13,1.848679
1,people,10,0.877912
2,friends,9,1.279314
3,would,7,0.569055


In [91]:
# n most frequent
n = 30

fig = px.bar(
    data_frame=f_words_df[:n], x='words', y='freq', color='freq',
    text='tfidf_score_sum', labels={'x': 'words', 'y': 'frequency'},
    title='Frequency of words seen in <b> Flagged </b>records (with TFIDF in parentheses)' )

fig.update_xaxes(tickangle= -45)  
fig.update_traces(
    texttemplate='%{text:.2f}', textposition='outside',
     textfont_size=8)
     
fig.show()

In [92]:
fig = px.bar(
    data_frame=n_words_df[:n], x='words', y='freq', color='freq',
    text='tfidf_score_sum', labels={'x': 'words', 'y': 'frequency'},
    title='Frequency of words seen in <b> Flagged </b>records (with TFIDF in parentheses)')

fig.update_xaxes(tickangle= -45)  
fig.update_traces(
    texttemplate='%{text:.2f}', textposition='outside',
     textfont_size=8)
     
fig.show()

In [94]:
merged_df

,words,freq_f,tfidf_score_sum_f,freq_n,tfidf_score_sum_n
0,friend,13,1.848679,9.0,2.241216
1,people,10,0.877912,10.0,2.230480
2,friends,9,1.279314,15.0,3.921829
3,would,7,0.569055,4.0,0.877480
4,going,6,1.168151,5.0,1.022397
...,...,...,...,...,...
343,spent,1,0.170482,NaN,NaN
344,nights,1,0.170482,NaN,NaN
345,letting,1,0.170482,2.0,0.705452
346,vent,1,0.170482,NaN,NaN


In [93]:
n_words_df = n_words_df.add_suffix('_n')
n_words_df.rename(columns={'words_n': 'words'}, inplace=True)

f_words_df = f_words_df.add_suffix('_f')
f_words_df.rename(columns={'words_f': 'words'}, inplace=True)

merged_df = merge(f_words_df, n_words_df)

In [95]:
merged_df_sample = merged_df[:n]
fig = go.Figure(data=[
    go.Bar(name='Flagged', x=merged_df_sample.words, y=merged_df_sample.freq_f, text=merged_df_sample.freq_f, marker_color='#BA0F30'),
    go.Bar(name='Not Flagged', x=merged_df_sample.words, y=merged_df_sample.freq_n, text=merged_df_sample.freq_n, marker_color='#98D7C2')
])


fig.update_xaxes(tickangle= -45)  
fig.update_traces(
    texttemplate='%{text:d}', textposition='outside',
     textfont_size=8)

# Change the bar mode
fig.update_layout(barmode='group', title_text='most frequent Flagged/not flagged words count comparison')
fig.show()


**Naive Bayes**

In [96]:
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()

In [97]:
transformed_data, words = get_tfidf_words_and_array(df.loc[:, 'cleaned_text'])
y = df.loc[:, 'class']

In [98]:
scores = cross_val_score(nb, transformed_data, y, cv=3)
scores

array([0.44444444, 0.37037037, 0.69230769])

In [99]:
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier()
scores = cross_val_score(random_forest, transformed_data, y, cv=3)
scores

array([0.7037037 , 0.66666667, 0.73076923])

## **Deep Learning**

In [100]:
import tensorflow as tf
import keras

from tensorflow.keras import layers
from sklearn.model_selection import KFold
from keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, LSTM, Dense, SimpleRNN, BatchNormalization

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [101]:
text = df.loc[:, 'response_text'].values
pad_type = 'post'
trunc_type='post'

# Tokenize our training data
tokenizer = Tokenizer()
texts = [str (item) for item in df.loc[:, 'response_text']]
tokenizer.fit_on_texts(texts)

# word index = {'word': idx}
word_index = tokenizer.word_index

# Encode into sequences
encoded_seqs = tokenizer.texts_to_sequences(texts)

In [102]:
padded_lengths = [len(encoded_seqs_i) for encoded_seqs_i in encoded_seqs]
px.histogram(padded_lengths)

In [103]:
max_len = 60
padded_texts = pad_sequences(encoded_seqs, padding=pad_type, truncating=trunc_type, maxlen=max_len)

In [104]:
# Output the results of our work
random_i = np.random.randint(len(df))

print("Encoded sample:", encoded_seqs[random_i])
print("Padded sample:", padded_texts[random_i])
print("Padded shape:", padded_texts.shape)
print("sequences data type:", type(encoded_seqs))
print("Padded  sequences data type:", type(padded_texts))

Encoded sample: [426, 17, 427, 172, 253, 100, 428, 429, 254, 7, 1, 430, 49, 431, 26, 25, 9, 63, 432, 2, 176, 13, 16, 200, 91, 255, 27, 433, 13, 3, 16, 434, 435, 436, 6, 13, 25, 114, 10, 437, 79, 26, 134, 69, 438, 32, 155, 5, 439, 6, 8, 256]
Padded sample: [426  17 427 172 253 100 428 429 254   7   1 430  49 431  26  25   9  63
 432   2 176  13  16 200  91 255  27 433  13   3  16 434 435 436   6  13
  25 114  10 437  79  26 134  69 438  32 155   5 439   6   8 256   0   0
   0   0   0   0   0   0]
Padded shape: (80, 60)
sequences data type: <class 'list'>
Padded  sequences data type: <class 'numpy.ndarray'>


**A simple RNN**

For an RNN, we need to have a one-hot-encoded input. We can also use an embedding layer in the first layer of our sequential mode. But I chose to use 0-1 vectors to see how a very basic model works on this dataset.

In [105]:
num_features = len(word_index) + 1

In [106]:
def to_categorical_tensor(x, num_classes=num_features, max_len=max_len):
    """
    x: [0, 1, 2, 3]
    output: tensor of one hot encoded text
    with the shape of (max sequence length, number of features) 
    """
    if type(x) != list: a = list(x)
    a = tf.keras.utils.to_categorical(x, num_classes)
    a = tf.constant(a, shape=[max_len, num_classes])
    return a

sample = to_categorical_tensor(x= padded_texts[0])
print(sample.shape)

(60, 678)


In [107]:
# one-hot-encoding
X = [to_categorical_tensor(padded_texts[i]) for i in range(len(df))]

# converting into an array
X_final = np.reshape(X, (len(X), max_len, num_features))

In [108]:
labels = np.array([1 if label=='flagged' else 0 for label in df.loc[:, 'class']])

In [109]:
def create_simpleRNN(rnn_nodes=32):

    model = Sequential([
        SimpleRNN(32, input_shape=(max_len, num_features)),
        BatchNormalization(),
        Dense(1, activation="sigmoid")
        ])
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

    return model

model = create_simpleRNN()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 32)                22752     
                                                                 
 batch_normalization (BatchN  (None, 32)               128       
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 22,913
Trainable params: 22,849
Non-trainable params: 64
_________________________________________________________________


In [110]:
n_epochs = 3
n_splits = 3
batch_s = 32 # the default is 32
mean_val_acc_models = {}

def evaluate(X, Y, model_, n_epochs=3, n_splits=3, batch_s=32):
    """
    Evaluates each model with cross validation
    """
    history = {}
    kfold = KFold(n_splits)
    splits = kfold.split(X)

    for i, (train_index, val_index) in enumerate(splits):

        print(f'\nfold {i+1}')
        model = model_()

        train_x = X[train_index]
        train_y = Y[train_index]

        val_x = X[val_index]
        val_y = Y[val_index]

        history[f'fold {i}'] = model.fit(
            train_x, train_y, epochs=n_epochs, batch_size=batch_s, verbose=2,
             validation_data=(val_x, val_y)).history
    mean_val_acc = [np.mean(hist['val_accuracy']) for hist in history.values()]
    return mean_val_acc

In [111]:
mean_val_acc_models['SimpleRNN'] = evaluate(X=X_final, Y=labels, model_=create_simpleRNN)


fold 1
Epoch 1/3
2/2 - 8s - loss: 0.9477 - accuracy: 0.4717 - val_loss: 0.6958 - val_accuracy: 0.4074 - 8s/epoch - 4s/step
Epoch 2/3
2/2 - 0s - loss: 0.5647 - accuracy: 0.7170 - val_loss: 0.6890 - val_accuracy: 0.5556 - 111ms/epoch - 56ms/step
Epoch 3/3
2/2 - 0s - loss: 0.4424 - accuracy: 0.8302 - val_loss: 0.6857 - val_accuracy: 0.5556 - 125ms/epoch - 63ms/step

fold 2
Epoch 1/3
2/2 - 2s - loss: 1.0897 - accuracy: 0.3208 - val_loss: 0.9533 - val_accuracy: 0.1481 - 2s/epoch - 1s/step
Epoch 2/3
2/2 - 0s - loss: 0.7817 - accuracy: 0.3774 - val_loss: 0.9333 - val_accuracy: 0.1481 - 334ms/epoch - 167ms/step
Epoch 3/3
2/2 - 0s - loss: 0.6656 - accuracy: 0.4528 - val_loss: 0.9068 - val_accuracy: 0.1111 - 470ms/epoch - 235ms/step

fold 3
Epoch 1/3
2/2 - 4s - loss: 0.6783 - accuracy: 0.5000 - val_loss: 0.6293 - val_accuracy: 0.7308 - 4s/epoch - 2s/step
Epoch 2/3
2/2 - 0s - loss: 0.5652 - accuracy: 0.7593 - val_loss: 0.6239 - val_accuracy: 0.7308 - 226ms/epoch - 113ms/step
Epoch 3/3
2/2 - 0s -

**GRU**

In [112]:
def create_GRU(embedding_dim=32, gru_d=32):

    model = Sequential([
        Embedding(input_dim=num_features, output_dim=embedding_dim, input_length=max_len),
        GRU(gru_d),
        BatchNormalization(),
        Dense(1, activation="sigmoid")
        ])
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

    return model

model = create_GRU()
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 60, 32)            21696     
                                                                 
 gru (GRU)                   (None, 32)                6336      
                                                                 
 batch_normalization_4 (Batc  (None, 32)               128       
 hNormalization)                                                 
                                                                 
 dense_4 (Dense)             (None, 1)                 33        
                                                                 
Total params: 28,193
Trainable params: 28,129
Non-trainable params: 64
_________________________________________________________________


In [113]:
mean_val_acc_models['GRU'] = evaluate(X=padded_texts, Y=labels, model_=create_GRU)


fold 1
Epoch 1/3
2/2 - 6s - loss: 0.7362 - accuracy: 0.2264 - val_loss: 0.6917 - val_accuracy: 0.5556 - 6s/epoch - 3s/step
Epoch 2/3
2/2 - 0s - loss: 0.6771 - accuracy: 0.7736 - val_loss: 0.6890 - val_accuracy: 0.5926 - 485ms/epoch - 242ms/step
Epoch 3/3
2/2 - 0s - loss: 0.6315 - accuracy: 0.7925 - val_loss: 0.6870 - val_accuracy: 0.5926 - 422ms/epoch - 211ms/step

fold 2
Epoch 1/3


2/2 - 6s - loss: 0.7161 - accuracy: 0.3585 - val_loss: 0.7022 - val_accuracy: 0.1852 - 6s/epoch - 3s/step
Epoch 2/3
2/2 - 1s - loss: 0.7018 - accuracy: 0.3774 - val_loss: 0.6920 - val_accuracy: 0.7778 - 562ms/epoch - 281ms/step
Epoch 3/3
2/2 - 1s - loss: 0.6734 - accuracy: 0.4717 - val_loss: 0.6826 - val_accuracy: 0.7778 - 689ms/epoch - 344ms/step

fold 3
Epoch 1/3


2/2 - 4s - loss: 0.7272 - accuracy: 0.2963 - val_loss: 0.6894 - val_accuracy: 0.6923 - 4s/epoch - 2s/step
Epoch 2/3
2/2 - 0s - loss: 0.6785 - accuracy: 0.7222 - val_loss: 0.6804 - val_accuracy: 0.6923 - 421ms/epoch - 210ms/step
Epoch 3/3
2/2 - 0s - loss: 0.6435 - accuracy: 0.7407 - val_loss: 0.6736 - val_accuracy: 0.6923 - 351ms/epoch - 176ms/step


# **Long Short-term Memory (LSTM)**

In [114]:
def create_LSTM(embedding_dim=32, lstm_u=32):

    model = Sequential([
        Embedding(input_dim=num_features, output_dim=embedding_dim, input_length=max_len),
        LSTM(lstm_u),
        BatchNormalization(),
        Dense(1, activation="sigmoid")
        ])
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

    return model

model = create_GRU()
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 60, 32)            21696     
                                                                 
 gru_4 (GRU)                 (None, 32)                6336      
                                                                 
 batch_normalization_8 (Batc  (None, 32)               128       
 hNormalization)                                                 
                                                                 
 dense_8 (Dense)             (None, 1)                 33        
                                                                 
Total params: 28,193
Trainable params: 28,129
Non-trainable params: 64
_________________________________________________________________


In [115]:
mean_val_acc_models['LSTM'] = evaluate(X=padded_texts, Y=labels, model_=create_GRU)


fold 1
Epoch 1/3
2/2 - 6s - loss: 0.6721 - accuracy: 0.7736 - val_loss: 0.6895 - val_accuracy: 0.5556 - 6s/epoch - 3s/step
Epoch 2/3
2/2 - 0s - loss: 0.6266 - accuracy: 0.7925 - val_loss: 0.6872 - val_accuracy: 0.5926 - 436ms/epoch - 218ms/step
Epoch 3/3
2/2 - 0s - loss: 0.6051 - accuracy: 0.7736 - val_loss: 0.6852 - val_accuracy: 0.5926 - 407ms/epoch - 203ms/step

fold 2
Epoch 1/3
2/2 - 8s - loss: 0.6796 - accuracy: 0.6792 - val_loss: 0.6886 - val_accuracy: 0.7778 - 8s/epoch - 4s/step
Epoch 2/3
2/2 - 2s - loss: 0.6724 - accuracy: 0.6226 - val_loss: 0.6803 - val_accuracy: 0.7778 - 2s/epoch - 886ms/step
Epoch 3/3
2/2 - 1s - loss: 0.6521 - accuracy: 0.6415 - val_loss: 0.6788 - val_accuracy: 0.7778 - 942ms/epoch - 471ms/step

fold 3
Epoch 1/3
2/2 - 4s - loss: 0.7235 - accuracy: 0.2963 - val_loss: 0.6944 - val_accuracy: 0.3462 - 4s/epoch - 2s/step
Epoch 2/3
2/2 - 0s - loss: 0.7008 - accuracy: 0.5370 - val_loss: 0.6878 - val_accuracy: 0.6923 - 370ms/epoch - 185ms/step
Epoch 3/3
2/2 - 0s - 

In [116]:
mean_val_acc_models

{'SimpleRNN': [0.5061728556950887, 0.1358024701476097, 0.7307692170143127],
 'GRU': [0.5802469253540039, 0.5802469203869501, 0.692307710647583],
 'LSTM': [0.5802469253540039, 0.7777777910232544, 0.5769230922063192]}